Se construirá un data set de los modismos con las columnas, palabra, significado y ejemplo

In [6]:
import pandas as pd
import re

In [9]:
path_file_academia = "AcademiaColombianaLengua/diccionario_colombianismos.txt"

data = []
current_entry = {}

with open(path_file_academia, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line:
            continue

        if line.startswith("**PALABRA:**"):
            if 'palabra' in current_entry:
                # Append previous entry if a new one starts
                data.append(current_entry)
            current_entry = {"palabra": line.replace("**PALABRA:**", "").strip()}
        elif line.startswith("**SIGNIFICADO:**"):
            current_entry["significado"] = line.replace("**SIGNIFICADO:**", "").strip()
        elif line.startswith("**EJEMPLO:**"):
            current_entry["ejemplo"] = line.replace("**EJEMPLO:**", "").strip()

# Add the last entry
if current_entry:
    data.append(current_entry)

df = pd.DataFrame(data)

# Display the first few rows of the dataframe
df.head()

,palabra,significado,ejemplo
0,abajeño,Propio o nativo de las costas o de las tierras...,Empezaron cultivos de tabaco y a elaborar un d...
1,abalear,Disparar a alguien o a algo de manera repetida...,El funcionario resultó ileso a pesar de que el...
2,abaleo,Situación en la que hay disparos repetidos que...,Por la calle doce se oía un tremendo abaleo y ...
3,abanicar,"En el beisbol, fallar el bateador al no tocar ...",El bateador se ponchó abanicando el tercer lan...
4,abanico,Aparato eléctrico que crea una corriente de ai...,No podía dormir la siesta de la tarde sin el a...


In [10]:
df.shape

(5833, 3)

Con este data set tenemos 5833 modismos

In [ ]:
path_file_CaroCuervo = "Lexicc - CaroCuervo\\Diccionario_breve_de_Colombiaismos_slown_text.txt"

data_caro = []
# Regex to find the word, definition and example
# It handles multiple definitions in the same line
pattern = re.compile(r"-\s\*\*(?P<palabra>[^|*]+)(?:\|\|.*?)?\*\*.*?(?=\d+\.|\s[A-ZÁ-ÚÑ])(?P<definicion>.*?)(?:\s\*(?P<ejemplo>.*?)\*(?:\s\(.*?\))?\s*$|$)")


with open(path_file_CaroCuervo, "r", encoding="utf-8") as f:
    for line in f:
        line = line.strip()
        if not line.startswith("- **"):
            continue
        
        # Split the line by numbered definitions like "2.", "3."
        parts = re.split(r'\s\*\*\d+\.\*\*\s', line)
        
        base_palabra_match = re.match(r'-\s\*\*(.*?)\*\*', parts[0])
        if not base_palabra_match:
            continue
        
        base_palabra = base_palabra_match.group(1).split('||')[0].strip()

        for i, part in enumerate(parts):
            content = part
            if i == 0:
                # For the first part, remove the initial "- **palabra**"
                content = re.sub(r'-\s\*\*.*?\*\*\s*', '', part)

            # Clean up content
            # Remove grammatical tags like 'tr.', 'm.', 'adj.'
            content = re.sub(r'\b(tr|intr|prnl|m|f|adj|adv|interj|sust|coloq|despect|obsol|pop|rur|vulg)\.\s', '', content)
            # Remove regional tags like '*Boy., Cund.*'
            content = re.sub(r'\*([A-Z][a-z]+(?:\.\s[A-Z][a-z]+)*\.)\*', '', content)
            content = content.strip()

            # Separate definition and example
            ejemplo_match = re.search(r'\s\*(.*?)\*.*$', content)
            
            definicion = content
            ejemplo = None

            if ejemplo_match:
                ejemplo = ejemplo_match.group(1).strip()
                definicion = content[:ejemplo_match.start()].strip()

            if definicion:
                data_caro.append({
                    "palabra": base_palabra,
                    "significado": definicion,
                    "ejemplo": ejemplo
                })


df_caro = pd.DataFrame(data_caro)
df_caro.head()